In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors

# 注意一下，由于这个笔记本同时用于处理训练和测试数据（二者的处理办法是一致的），因此有些代码是选择性的

house_data = pd.read_csv("/home/mw/input/quant4533/ruc_Class25Q1_train.csv")
# house_data = pd.read_csv("/home/mw/input/quant4533/ruc_Class25Q1_test.csv")
county_data = pd.read_csv("/home/mw/input/quant4533/ruc_Class25Q1_details.csv")

# 准备房价数据，准备面积数据，计算单位面积的房价
house_data["建筑面积"]=house_data["建筑面积"].str.replace("㎡", "", regex=False).astype(float)
house_data["单位面积均价"]=house_data["价格"]/house_data["建筑面积"]

# 因为小区数据有重复的，所以把有重复的小区全部去掉
county_data=county_data.drop_duplicates(subset=["板块","名称"])

# 把小区数据merge到房屋数据上
county_data=county_data.rename(columns={"名称":"小区名称"})
house_data=pd.merge(house_data,county_data,on=["板块","小区名称"],how="left")

def self_fill_in(name1,name2):
    """
    这个函数的功能是，用merge进来的数据补充原数据的缺失值
    """
    house_data[name1]=house_data[name1].fillna(house_data[name2])

# 用小区的环线位置数据替代缺失样本
self_fill_in("环线","环线位置")

# 利用该板块的环线数据，填充“环线”数据
def get_mode(series):
    """
    这个函数的功能是，求一个序列的众数，如果没有就返回None
    """
    result = series.mode()
    return result[0] if not result.empty else None

def friend_fill_in(group,name):
    """
    这个函数的作用是，根据group分组，然后将组内的name填充给缺失值
    """
    house_data[name]=house_data.groupby(group)[name].transform(
        lambda x: x.fillna(get_mode(x)) if not get_mode(x) == None else x
    )

friend_fill_in("板块","环线")

friend_fill_in("小区名称","环线")

def KNN_fill_in(target,using):
    """
    这个函数的功能是，使用using的变量和KNN算法来填补target中的空缺值
    """
    global house_data
    non_missing=house_data[house_data[target].notnull()]
    missing=house_data[house_data[target].isnull()]

    features = non_missing[using]
    target_features = missing[using]

    model_knn = NearestNeighbors(n_neighbors=1)
    model_knn.fit(features)

    distances,indices=model_knn.kneighbors(target_features)

    for i, idx in enumerate(indices):
        missing.iloc[i, missing.columns.get_loc(target)] = non_missing.iloc[idx[0]][target]

    house_data = pd.concat([non_missing, missing]).sort_index()

KNN_fill_in("环线",["lon","lat"])

KNN_fill_in("房屋户型",["lon","lat","建筑面积"])

def split_and_merge(df,column_name):
    """
    这个函数的功能是：拆分指定列（如“户型”）中的字符串为多个字段（如房间数、厅数等），将拆分后的字段合并到原始数据框中。
    """
    # 定义拆分函数
    def split_layout(layout):
        # 使用正则表达式提取数字
        numbers = list(map(int, re.findall(r"\d+", layout)))
        # 确保提取了 4 个数字（房间、厅、厨房、卫生间）
        if len(numbers)==4:
            return pd.Series({
                "房间数": numbers[0],
                "厅数": numbers[1],
                "厨房数": numbers[2],
                "卫生间数": numbers[3],
            })
        elif len(numbers)==2:
            return pd.Series({
                "房间数": numbers[0],
                "厅数": 0,
                "厨房数": 0,
                "卫生间数": numbers[1],
            })
    
    # 拆分指定列
    df_split = df[column_name].apply(split_layout)
    
    # 合并拆分后的数据
    df_result = pd.concat([df, df_split], axis=1)
    
    return df_result


house_data = split_and_merge(house_data, "房屋户型")

# 为每个城市环线生成一个虚拟变量
house_data["城市_环线"] = house_data["城市_x"].astype(str) + "_" + house_data["环线"]
dummies = pd.get_dummies(house_data["城市_环线"], prefix="城市环线", drop_first=True)
house_data = pd.concat([house_data, dummies], axis=1)

# 为每个板块生成一个虚拟变量
house_data["区域_板块"] = house_data["区域"].astype(str)+"_"+house_data["板块"].astype(str)
dummies = pd.get_dummies(house_data["区域_板块"], prefix="区域板块",drop_first=True)
house_data = pd.concat([house_data, dummies], axis=1)

region_columns = house_data.columns[house_data.columns.str.startswith("区域板块")]
ring_columns = house_data.columns[house_data.columns.str.startswith("城市环线")]
house_data[["建筑面积","房间数","厅数","厨房数","卫生间数"]+region_columns.tolist()+ring_columns.tolist()].to_csv("/home/mw/project/using.csv",index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84133 entries, 0 to 84132
Data columns (total 57 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   城市_x     84133 non-null  int64  
 1   区域       84133 non-null  float64
 2   板块       84133 non-null  float64
 3   环线       42726 non-null  object 
 4   小区名称     84133 non-null  object 
 5   价格       84133 non-null  int64  
 6   房屋户型     83528 non-null  object 
 7   所在楼层     84133 non-null  object 
 8   建筑面积     84133 non-null  float64
 9   套内面积     25146 non-null  object 
 10  房屋朝向     84133 non-null  object 
 11  建筑结构_x   83528 non-null  object 
 12  装修情况     83528 non-null  object 
 13  梯户比例     82438 non-null  object 
 14  配备电梯     75818 non-null  object 
 15  别墅类型     749 non-null    object 
 16  交易时间     84133 non-null  object 
 17  交易权属     84133 non-null  object 
 18  上次交易     55180 non-null  object 
 19  房屋用途     84131 non-null  object 
 20  房屋年限     54351 non-null  object 
 21  产权所属     841

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 84133 entries, 0 to 84132
Data columns (total 57 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   城市_x     84133 non-null  int64  
 1   区域       84133 non-null  float64
 2   板块       84133 non-null  float64
 3   环线       84133 non-null  object 
 4   小区名称     84133 non-null  object 
 5   价格       84133 non-null  int64  
 6   房屋户型     83528 non-null  object 
 7   所在楼层     84133 non-null  object 
 8   建筑面积     84133 non-null  float64
 9   套内面积     25146 non-null  object 
 10  房屋朝向     84133 non-null  object 
 11  建筑结构_x   83528 non-null  object 
 12  装修情况     83528 non-null  object 
 13  梯户比例     82438 non-null  object 
 14  配备电梯     75818 non-null  object 
 15  别墅类型     749 non-null    object 
 16  交易时间     84133 non-null  object 
 17  交易权属     84133 non-null  object 
 18  上次交易     55180 non-null  object 
 19  房屋用途     84131 non-null  object 
 20  房屋年限     54351 non-null  object 
 21  产权所属     841

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
